<a href="https://colab.research.google.com/github/nisanuro/CNG483-Project1/blob/master/CNG483_Project1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn  import preprocessing
from google.colab import output
from google.colab import drive
import concurrent.futures
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def read_images(path):
    images = []

    for filename in os.listdir(path):
        img=cv2.imread(os.path.join(image_path, filename),cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(img)

    return images

In [0]:
"""
def histogram(image, b):
    bins = []
    width = int(256/b)

    for low in range(0, b * width, width):
        bins.append((low, low+width))

    hist = np.zeros(b).astype('int32')
    image = np.array(image).flatten()

    for value in image:
        for i in range(0, len(bins)):
            if (bins[i][0] <= value < bins[i][1]):
                hist[i] += 1
    
    return hist
"""

def histogram(image,b):
    bins = np.array([i for i in range(0, 256, int(256/b))])
    image = np.array(image).flatten()
    inds = np.digitize(image,bins, right=False)
    temp = np.array([(inds[i]-1) for i in range(0, len(inds))])
    count = np.bincount(temp)
    
    hist = np.array([count[i] for i in range(0, len(count), int(256/b))])
    if len(hist) != b:
        z = np.zeros(b-len(hist)).astype('int64')
        hist = np.concatenate((hist, z))

    #print("hist: ",  len(hist), "hist shape: " , hist.shape)
    #count = count[::int(256/b)]
    #print(count)
    #print(bins)
    return hist
  

In [0]:
def ndmesh(*xi,**kwargs):
    if len(xi) < 2:
        msg = 'meshgrid() takes 2 or more arguments (%d given)' % int(len(xi) > 0)
        raise ValueError(msg)

    args = np.atleast_1d(*xi)
    ndim = len(args)
    copy_ = kwargs.get('copy', True)

    s0 = (1,) * ndim
    output = [x.reshape(s0[:i] + (-1,) + s0[i + 1::]) for i, x in enumerate(args)]

    shape = [x.size for x in output]

    # Return the full N-D matrix (not only the 1-D vector)
    if copy_:
        mult_fact = np.ones(shape, dtype=int)
        return [x * mult_fact for x in output]
    else:
        return np.broadcast_arrays(*output)

In [0]:
def color_histogram(img, b):
    chans = cv2.split(img)
    colors = ("b", "g", "r")

    features = []
   
    for (chan, color) in zip(chans, colors):
        if color == 'b':
            bf = histogram(chan, b)
        elif color == 'g':
            gf = histogram(chan, b)
        else:
            rf = histogram(chan, b)
    """
    # combination of blue and red channel bins
    temp = np.transpose([np.tile(bf, len(gf)), np.repeat(gf, len(bf))])
    
    blue_green_hist = []    
    for i in temp:
        blue_green_hist.append(sum(i))

    # same operation with red channel bins
    temp = np.transpose([np.tile(rf, len(blue_green_hist)), np.repeat(blue_green_hist, len(rf))])
    """
    #print("bf:  ", bf.shape)
    #print(bf)
    temp = np.vstack((ndmesh(bf,gf,rf))).reshape(3,-1).T 
    #print("temp:", temp.shape)
    #print(temp)
    #color_hist = []    
    color_hist = temp.sum(axis=1)
    #for i in temp:
    #    color_hist.append(sum(i))
    color_hist = np.array(color_hist)
    #print("color hist: ", color_hist.shape)
    
    return color_hist

In [0]:
def split_image_into_grids(image, grid):

    grids = []

    height=image.shape[0]
    width=image.shape[1]

    M = height//grid
    N = width//grid

    for y in range(0, height, M):
        for x in range(0, width, N):
            y1 = y + M
            x1 = x + N
            grids.append(image[y:y+M, x:x+N])


    #cv2.rectangle(image, (x, y), (x1, y1), (0, 255, 0))

    #os.chdir("/content/drive/My Drive/SaveDeneme")
    #cv2.imwrite("im" + str(i) + '_' + str(x) + '_' + str(y)+".png",tiles)
    '''
    plt.figure()
    plt.imshow(grid) 
    plt.show()''' 

    return np.array(grids)

     

In [0]:
def create_dataset(path):
    print("Loading dataset")
    class_paths = []
    images = []
    labels = []

    with os.scandir(path) as itr: 
        for subdir in itr : 
            if subdir.is_dir():
                p = path + '/' + subdir.name
                class_paths.append(p)
    c = 0
    for p in class_paths:
        #print(p)
        for filename in os.listdir(p):
            c += 1
            print("data: ", c)
            if filename.endswith(".jpg"):
                labels.append(os.path.basename(os.path.normpath(p))) 
                img = cv2.imread(os.path.join(p, filename))
                if img is not None:
                    images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                 
            elif filename.endswith(".npy"):
                labels.append(os.path.basename(os.path.normpath(p))) 
                img = np.load(os.path.join(p, filename))
                images.append(img)
    return images, labels


In [0]:
def threaded_feature_extraction(imageslabels, grid, bin, isRGB, datasetType):
    print("feature extraction")
    features = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futuretoimage = {executor.submit(feature_extract_single, imagelabel, grid, bin, isRGB): imagelabel for imagelabel in imageslabels}
        counter = 0
        for future in concurrent.futures.as_completed(futuretoimage):
            counter += 1

            if datasetType == 0:    # Training Set
                path = '/content/CNG483-Project/Features/TrainingSet' + '/' + future.result()[1]
            elif datasetType == 1:  # Validation Set
                path = '/content/CNG483-Project/Features/ValidationSet' + '/' + future.result()[1]
            elif datasetType == 2:  # Test Set
                path = '/content/CNG483-Project/Features/TestSet' + '/' + future.result()[1]                    
            
            if not os.path.exists(path):
                try: 
                    os.makedirs(path, exist_ok = True) 
                    print("Directory created successfully:  ", path) 
                except OSError as error: 
                    print("Directory can not be created")
            else:
                print("Directory already exists:   ", path)
            path =  path + '/' + str(counter)

            temp = np.array(future.result()[0])
            temp = temp.reshape(1, len(temp))
            normalized = preprocessing.normalize(temp, norm='l1', axis=0, copy=False)
            np.save(path, np.array(normalized))
            
            print("IMAGE : ", counter)
    #return features

def feature_extract_single(imagelabel, grid, b, isRGB):
    image = imagelabel[0]
    label = imagelabel[1]
    #print(label)
    i = cv2.resize(image, (256, 256))
    i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
    grid_hists = []
    grids = split_image_into_grids(i, grid)
    #print("grids: ")
    #print(len(grids))
    for i in grids:
        if(isRGB):
            grid_hists.append(color_histogram(i, b))
        else:
            i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
            grid_hists.append(histogram(i, b))
    '''
    # Concatenation
    grids_flat = np.array(grid_hists).flatten()
    '''

    # Sum
    if grid != 1:
        grids_flat = [ sum(x) for x in zip(*grid_hists) ]
        grids_flat = np.array(grids_flat).flatten()
        #print("grids: ", len(grids_flat))
        
    else:
        grids_flat = np.array(grid_hists).flatten()
        #print("grids: ", len(grid_hists))
    
    #grids_flat = preprocessing.normalize(grids_flat.reshape(1, -1), norm='l1')
    return (grids_flat, label)

    


In [9]:
"""def feature_extraction(images, grid, bin, isRGB):
    print("feature extraction")
    features = []
    size = len(images)
    for i, j in zip(images, range(size)):
        print(j)
        i = cv2.resize(i, (256, 256))
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        grids = split_image_into_grids(i, grid)

        grid_hists = []

        for i in grids:
            if(isRGB):
                grid_hists.append(color_histogram(i, bin))
            else:
                grid_hists.append(histogram(i, bin))
        
        # Concatenation
        grids_flat = np.array(grid_hists).flatten()
        
        '''
        # Sum
        grids_flat = np.array(grid_hists)
        grids_flat = np.sum(grids_flat, axis=0)
        '''
            
        features.append(grids_flat)
        
    return features"""


'def feature_extraction(images, grid, bin, isRGB):\n    print("feature extraction")\n    features = []\n    size = len(images)\n    for i, j in zip(images, range(size)):\n        print(j)\n        i = cv2.resize(i, (256, 256))\n        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)\n        grids = split_image_into_grids(i, grid)\n\n        grid_hists = []\n\n        for i in grids:\n            if(isRGB):\n                grid_hists.append(color_histogram(i, bin))\n            else:\n                grid_hists.append(histogram(i, bin))\n        \n        # Concatenation\n        grids_flat = np.array(grid_hists).flatten()\n        \n        \'\'\'\n        # Sum\n        grids_flat = np.array(grid_hists)\n        grids_flat = np.sum(grids_flat, axis=0)\n        \'\'\'\n            \n        features.append(grids_flat)\n        \n    return features'

In [0]:
def classification(trainI, trainL, testI, testL, k):
    print("classification")
    model = KNeighborsClassifier(n_neighbors=k, metric='euclidean', algorithm='brute', n_jobs=-1)
    model.fit(trainI, trainL)
    acc = model.score(testI, testL)
    print(acc)

In [11]:
if __name__ == "__main__":
    
    test_path = "/content/drive/My Drive/CNG483-Project 1/TestSet"
    train_path = "/content/drive/My Drive/CNG483-Project 1/TrainingSet"
    validation_path = "/content/drive/My Drive/CNG483-Project 1/ValidationSet"

    
    # testI, testL = create_dataset(test_path)
    trainI, trainL = create_dataset(train_path)
    validationI, validationL = create_dataset(validation_path)
    
    # grid --> 1, 2, 4
    # bins --> 1, 128, 256
    # k --> 1, 5, 10
    # isRGB --> True, False

    grid = 1
    bins = 128
    isRGB = True
    k = 10
    
    imagelabels = []
    for i,l in zip(trainI, trainL):
        imagelabels.append((i,l))
    threaded_feature_extraction(imagelabels, grid, bins, isRGB, 0)

    imagelabels = []
    for i,l in zip(validationI, validationL):
        imagelabels.append((i,l))
    
    threaded_feature_extraction(imagelabels, grid, bins, isRGB, 1)
    

Loading dataset
data:  1
data:  2
data:  3
data:  4
data:  5
data:  6
data:  7
data:  8
data:  9
data:  10
data:  11
data:  12
data:  13
data:  14
data:  15
data:  16
data:  17
data:  18
data:  19
data:  20
data:  21
data:  22
data:  23
data:  24
data:  25
data:  26
data:  27
data:  28
data:  29
data:  30
data:  31
data:  32
data:  33
data:  34
data:  35
data:  36
data:  37
data:  38
data:  39
data:  40
data:  41
data:  42
data:  43
data:  44
data:  45
data:  46
data:  47
data:  48
data:  49
data:  50
data:  51
data:  52
data:  53
data:  54
data:  55
data:  56
data:  57
data:  58
data:  59
data:  60
data:  61
data:  62
data:  63
data:  64
data:  65
data:  66
data:  67
data:  68
data:  69
data:  70
data:  71
data:  72
data:  73
data:  74
data:  75
data:  76
data:  77
data:  78
data:  79
data:  80
data:  81
data:  82
data:  83
data:  84
data:  85
data:  86
data:  87
data:  88
data:  89
data:  90
data:  91
data:  92
data:  93
data:  94
data:  95
data:  96
data:  97
data:  98
data:  99
dat

In [0]:
    '''
    imagelabels = []
    for i,l in zip(trainI, trainL):
        imagelabels.append((i,l))
    trainI_f = threaded_feature_extraction(imagelabels, grid, bins, isRGB)
    #trainI_f = threaded_feature_extraction(trainI, grid, bins, isRGB)
    trainf = []
    trainl = []
    for (i,l) in trainI_f:
        trainf.append(i)
        trainl.append(l)

    imagelabels = []
    for i,l in zip(validationI, validationL):
        imagelabels.append((i,l))
    
    validationI_f = threaded_feature_extraction(imagelabels, grid, bins, isRGB)
    validationf = []
    validationl = []
    for (i,l) in validationI_f:      
        validationf.append(i)
        validationl.append(l)

    #validationI_f = feature_extraction(validationI, grid, bins, isRGB)
    
    trainf = np.array(trainf)
    trainl = np.array(trainl)
    validationf = np.array(validationf)
    validationl = np.array(validationl)
    '''

In [12]:
    #test_path = "/content/drive/My Drive/CNG483-Project 1/TestSet"
    train_path = "/content/CNG483-Project/Features/TrainingSet"
    validation_path = "/content/CNG483-Project/Features/ValidationSet"

    
    # testI, testL = create_dataset(test_path)
    trainF, trainL = create_dataset(train_path)
    validationF, validationL = create_dataset(validation_path)

Loading dataset
data:  1
data:  2
data:  3
data:  4
data:  5
data:  6
data:  7
data:  8
data:  9
data:  10
data:  11
data:  12
data:  13
data:  14
data:  15
data:  16
data:  17
data:  18
data:  19
data:  20
data:  21
data:  22
data:  23
data:  24
data:  25
data:  26
data:  27
data:  28
data:  29
data:  30
data:  31
data:  32
data:  33
data:  34
data:  35
data:  36
data:  37
data:  38
data:  39
data:  40
data:  41
data:  42
data:  43
data:  44
data:  45
data:  46
data:  47
data:  48
data:  49
data:  50
data:  51
data:  52
data:  53
data:  54
data:  55
data:  56
data:  57
data:  58
data:  59
data:  60
data:  61
data:  62
data:  63
data:  64
data:  65
data:  66
data:  67
data:  68
data:  69
data:  70
data:  71
data:  72
data:  73
data:  74
data:  75
data:  76
data:  77
data:  78
data:  79
data:  80
data:  81
data:  82
data:  83
data:  84
data:  85
data:  86
data:  87
data:  88
data:  89
data:  90
data:  91
data:  92
data:  93
data:  94
data:  95
data:  96
data:  97
data:  98
data:  99
dat

In [1]:
np.array(trainF).shape


NameError: ignored

In [15]:
    print(trainF.dtype)
    print(validationF.dtype)
    print(trainL.dtype)
    print(validationL.dtype)

AttributeError: ignored

In [13]:
    classification(trainF, trainL, validationF, validationL, k)
    output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

classification


ValueError: ignored